In [1]:
import numpy as np

from keras import backend as K

import tensorflow as tf

Using TensorFlow backend.


## experiment

In [2]:
sess = K.get_session()

sess_tf = tf.Session()

sess, sess_tf

(<tensorflow.python.client.session.Session at 0x7f93f71ea4a8>,
 <tensorflow.python.client.session.Session at 0x7f93f71da8d0>)

`K.variable`

In [16]:
kvar = K.variable([[1, 2], [3, 4]])
kvar

<tf.Variable 'Variable_4:0' shape=(2, 2) dtype=float32_ref>

`tf.Variable`

In [17]:
tfvar = tf.Variable([[1, 2], [3, 4]])
tfvar

<tf.Variable 'Variable_5:0' shape=(2, 2) dtype=int32_ref>

In [19]:
K.eval(kvar)

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

In [20]:
K.eval(tfvar)

array([[1, 2],
       [3, 4]], dtype=int32)

In [21]:
kvar.eval(session=sess)

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

In [22]:
tfvar.eval(session=sess)

array([[1, 2],
       [3, 4]], dtype=int32)

In [23]:
kvar.eval(session=sess_tf)

FailedPreconditionError: Attempting to use uninitialized value Variable_4
	 [[Node: _retval_Variable_4_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_4)]]

In [24]:
tfvar.eval(session=sess_tf)

FailedPreconditionError: Attempting to use uninitialized value Variable_5
	 [[Node: _retval_Variable_5_0_0 = _Retval[T=DT_INT32, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_5)]]

if you set `sess_tf` as backend session, `K.eval(kvar)` also stops working 

In [25]:
K.set_session(sess_tf)

K.get_session() == sess_tf

True

In [26]:
K.eval(kvar)

FailedPreconditionError: Attempting to use uninitialized value Variable_4
	 [[Node: _retval_Variable_4_0_0 = _Retval[T=DT_FLOAT, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_4)]]

set back

In [27]:
K.set_session(sess)
K.eval(kvar)

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

need to initialize to make above erorrs go away

In [29]:
init = tf.global_variables_initializer()
sess_tf.run(init)

In [30]:
kvar.eval(session=sess_tf)

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

In [31]:
tfvar.eval(session=sess_tf)

array([[1, 2],
       [3, 4]], dtype=int32)

In [32]:
K.set_session(sess_tf)

K.get_session() == sess_tf

True

In [33]:
K.eval(kvar)

array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32)

## inspect functions

In [34]:
from inspect import getsourcelines
def print_source(func):
    print(' '.join(getsourcelines(func)[0]))

In [39]:
print_source(K.get_session)

def get_session():
     """Returns the TF session to be used by the backend.
 
     If a default TensorFlow session is available, we will return it.
 
     Else, we will return the global Keras session.
 
     If no global Keras session exists at this point:
     we will create a new global session.
 
     Note that you can manually set the global session
     via `K.set_session(sess)`.
 
     # Returns
         A TensorFlow session.
     """
     global _SESSION
     if tf.get_default_session() is not None:
         session = tf.get_default_session()
     else:
         if _SESSION is None:
             if not os.environ.get('OMP_NUM_THREADS'):
                 config = tf.ConfigProto(allow_soft_placement=True)
             else:
                 num_thread = int(os.environ.get('OMP_NUM_THREADS'))
                 config = tf.ConfigProto(intra_op_parallelism_threads=num_thread,
                                         allow_soft_placement=True)
             _SESSION = tf.Session(confi

In [35]:
K.eval

<function keras.backend.tensorflow_backend.eval>

In [36]:
print_source(K.eval)

def eval(x):
     """Evaluates the value of a variable.
 
     # Arguments
         x: A variable.
 
     # Returns
         A Numpy array.
 
     # Examples
     ```python
         >>> from keras import backend as K
         >>> kvar = K.variable(np.array([[1, 2], [3, 4]]), dtype='float32')
         >>> K.eval(kvar)
         array([[ 1.,  2.],
                [ 3.,  4.]], dtype=float32)
     ```
     """
     return to_dense(x).eval(session=get_session())



In [37]:
kvar.eval

<bound method Variable.eval of <tf.Variable 'Variable_4:0' shape=(2, 2) dtype=float32_ref>>

In [38]:
print_source(kvar.eval)

  def eval(self, session=None):
     """In a session, computes and returns the value of this variable.
 
     This is not a graph construction method, it does not add ops to the graph.
 
     This convenience method requires a session where the graph
     containing this variable has been launched. If no session is
     passed, the default session is used.  See @{tf.Session} for more
     information on launching a graph and on sessions.
 
     ```python
     v = tf.Variable([1, 2])
     init = tf.global_variables_initializer()
 
     with tf.Session() as sess:
         sess.run(init)
         # Usage passing the session explicitly.
         print(v.eval(sess))
         # Usage with the default session.  The 'with' block
         # above makes 'sess' the default session.
         print(v.eval())
     ```
 
     Args:
       session: The session to use to evaluate this variable. If
         none, the default session is used.
 
     Returns:
       A numpy `ndarray` with a copy of the va